# Example: Bag of Words, TF-IDF and PMI
In this example, we'll play around with simple collections of text data and explore how to create basic text embeddings using the Bag of Words model, Term Frequency-Inverse Document Frequency (TF-IDF), and Pointwise Mutual Information (PMI).

> __Learning Objectives:__
> 
> By the end of this example, you should be able to:
>
> Three learning objectives go here.

Let's get started!
___

## Summary
One concise, direct summary sentence goes here.

> __Key Takeaways:__
> 
> Three key takeaways go here.

One concise, direct concluding sentence goes here.
___